# 03. 다중 선형 회귀(Multivariable Linear regression)


앞서 배운 x가 1개인 선형 회귀를 `단순 선형 회귀(Simple Linear Regression)`이라고 합니다.
이번 챕터에서는 __다수의 x로부터 y를 예측하는 `다중 선형 회귀(Multivariable Linear Regression)`__에 대해서 이해합니다.


$H(x)=w_1x_1+w_2x_2+w_3x_3+b$

<img align='left' src='https://wikidocs.net/images/page/54841/%ED%9B%88%EB%A0%A8%EB%8D%B0%EC%9D%B4%ED%84%B0.PNG'>

In [3]:
import torch
import torch.nn as nn # 신경망구축을위한 데이터구조, 레이어, loss 등
import torch.nn.functional as F
import torch.optim as optim # 파라미터 최적화 알고리즘 (SGD 등)

In [4]:
torch.manual_seed(1) # 랜덤 시드 설정

위의 식에서 x가 3개 이므로 x를 3개 선언합니다.

In [14]:
# train data
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

이제 가중치 $w$와 편향 $b$를 선언합니다. 가중치 $w$도 3개 선언해주어야 합니다.

In [10]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

이제 가설, cost function, 옵티마이저를 선언한 후에 경사 하강법(GD)을 1,000회 반복합니다.

In [15]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    hypothesis = x1_train*w1 + x2_train*w2 + x3_train*w3 + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # 초기화
    cost.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2 {:.3f} w3 {:.3f} b {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))
    

Epoch    0/1000 w1: 0.294 w2 0.294 w3 0.297 b 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2 0.661 w3 0.676 b 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2 0.655 w3 0.677 b 0.008 Cost: 1.497607
Epoch  300/1000 w1: 0.684 w2 0.649 w3 0.677 b 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2 0.643 w3 0.678 b 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2 0.638 w3 0.678 b 0.009 Cost: 1.319511
Epoch  600/1000 w1: 0.699 w2 0.633 w3 0.679 b 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2 0.627 w3 0.679 b 0.009 Cost: 1.215696
Epoch  800/1000 w1: 0.709 w2 0.622 w3 0.679 b 0.009 Cost: 1.167818
Epoch  900/1000 w1: 0.713 w2 0.617 w3 0.680 b 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2 0.613 w3 0.680 b 0.009 Cost: 1.079378


***

## 벡터와 행렬 연산으로 바꾸기

위의 코드를 개선할 수 있는 부분이 있습니다. 이번에는 x의 개수가 3개였으니까 x1_train, x2_train, x3_train와 w1, w2, w3를 일일히 선언해주었습니다. 그런데 x의 개수가 1,000개라고 가정해봅시다. 위와 같은 방식을 고수할 경우 x_train1 ~ x_train1000을 전부 선언하고, w1 ~ w1000을 전부 선언해야 합니다. 다시 말해 x와 w 변수 선언만 총 합 2,000개를 해야합니다. 또한 가설을 선언하는 부분에서도 마찬가지로 x_train과 w의 곱셈이 이루어지는 항을 1,000개를 작성해야 합니다. 이는 굉장히 비효율적입니다.

이를 해결하기 위해 행렬 곱셈 연산(또는 벡터의 내적)을 사용합니다.
- 행렬의 곱셈 과정에서 이루어지는 벡터 연산을 `벡터의 내적(Dot Product)`이라고 합니다.


<img align='left' src='https://wikidocs.net/images/page/54841/%ED%96%89%EB%A0%AC%EA%B3%B1.PNG'>

위의 그림은 행렬 곱셈 연산 과정에서 벡터의 내적으로 1 × 7 + 2 × 9 + 3 × 11 = 58이 되는 과정을 보여줍니다.

이 행렬 연산이 어떻게 현재 배우고 있는 가설과 상관이 있다는 걸까요?
바로 가설을 벡터와 행렬 연산으로 표현할 수 있기 때문입니다.

### 1. 벡터 연산으로 이해하기

$H(x)=w_1x_1+w_2x_2+w_3x_3+b$

위 식은 아래와 같이 두 벡터의 내적으로 표현할 수 있습니다.

<img align='left' src='https://wikidocs.net/images/page/54841/%EB%82%B4%EC%A0%81.PNG'>

두 벡터를 각각 X와 W로 표현한다면, 가설은 다음과 같습니다.

$H(X)=XW$


x의 개수가 3개였음에도 이제는 X와 W라는 두 개의 변수로 표현된 것을 볼 수 있습니다.



### 2. 행렬 연산으로 이해하기

훈련 데이터를 살펴보고, 벡터와 행렬 연산을 통해 가설 H(X)를 표현해보겠습니다


<img align='left' src='https://wikidocs.net/images/page/54841/%ED%9B%88%EB%A0%A8%EB%8D%B0%EC%9D%B4%ED%84%B0.PNG'>

전체 훈련 데이터의 개수를 셀 수 있는 1개의 단위를 `샘플(sample)`이라고 합니다. 현재 샘플의 수는 총 5개입니다.
각 샘플에서 y를 결정하게 하는 각각의 독립 변수 x를 `특성(feature)`이라고 합니다. 현재 특성은 3개입니다.

이는 종속 변수 x들의 수가 (샘플의 수 × 특성의 수) = 15개임을 의미합니다. 종속 변수 x들을 (샘플의 수 × 특성의 수)의 크기를 가지는 하나의 행렬로 표현해봅시다. 그리고 이 행렬을 X라고 하겠습니다.

$\begin{bmatrix}
x11 & x12 & x13 \\\\ 
x21 & x22 & x23 \\\\ 
x31 & x32 & x33 \\\\ 
x41 & x42 & x43 \\\\ 
x51 & x52 & x53  
\end{bmatrix}$

그리고 여기에 가중치 w1,w2,w3을 원소로 하는 벡터를 W라 하고 이를 곱해보겠습니다.

$\begin{bmatrix}
x11 & x12 & x13 \\\\ 
x21 & x22 & x23 \\\\ 
x31 & x32 & x33 \\\\ 
x41 & x42 & x43 \\\\ 
x51 & x52 & x53  
\end{bmatrix}$
$\begin{bmatrix}
w1 \\\\ 
w2 \\\\ 
w3  
\end{bmatrix}$
$=$
$\begin{bmatrix}
x11w1 + x12w2 + x13w3 \\\\ 
x21w1 + x22w2 + x23w3 \\\\ 
x31w1 + x32w2 + x33w3 \\\\ 
x41w1 + x42w2 + x43w3 \\\\ 
x51w1 + x52w2 + x53w3  
\end{bmatrix}$

위의 식은 결과적으로 다음과 같습니다.

$H(X)=XW$

이 가설에 각 샘플에 더해지는 편향 $b$를 추가해봅시다. 샘플 수만큼의 차원을 가지는 편향 벡터 $B$를 만들어 더합니다.

$\begin{bmatrix}
x11 & x12 & x13 \\\\ 
x21 & x22 & x23 \\\\ 
x31 & x32 & x33 \\\\ 
x41 & x42 & x43 \\\\ 
x51 & x52 & x53  
\end{bmatrix}$
$\begin{bmatrix}
w1 \\\\ 
w2 \\\\ 
w3  
\end{bmatrix}$
$+$
$\begin{bmatrix}
b \\\\ 
b \\\\ 
b \\\\ 
b \\\\ 
b
\end{bmatrix}$
$=$
$\begin{bmatrix}
x11w1 + x12w2 + x13w3 + b \\\\ 
x21w1 + x22w2 + x23w3 + b \\\\ 
x31w1 + x32w2 + x33w3 + b \\\\ 
x41w1 + x42w2 + x43w3 + b \\\\ 
x51w1 + x52w2 + x53w3 + b  
\end{bmatrix}$

위의 식은 결과적으로 다음과 같습니다.

$H(X)=XW+B$


결과적으로 전체 훈련 데이터의 가설 연산을 3개의 변수만으로 표현하였습니다.
이와 같이 벡터와 행렬 연산은 식을 간단하게 해줄 뿐만 아니라 다수의 샘플의 병렬 연산이므로 속도의 이점을 가집니다.

이를 참고로 파이토치로 구현해봅시다.

***

## 행렬 연산을 고려하여 파이토치로 구현하기

이번에는 행렬 연산을 고려하여 파이토치로 재구현해보겠습니다.
이번에는 훈련 데이터 또한 행렬로 선언해야 합니다.

In [21]:
x_train = torch.FloatTensor([
    [73, 80, 75],
    [93, 88, 93],
    [89, 91, 90],
    [96, 98, 100],
    [73, 66, 70]
])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

이전에 x_train을 3개나 구현했던 것과 다르게 이번에는 x_train 하나에 모든 샘플을 전부 선언하였습니다. 다시 말해 (5 x 3) 행렬 X을 선언한 것입니다.

In [22]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


각각 (5 × 3) 행렬과 (5 × 1) 행렬(또는 벡터)의 크기를 가집니다.
이제 가중치 W와 편향 b를 선언합니다.

In [25]:
# 가중치와 편향 선언
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

여기서 주목할 점은 가중치 W의 크기가 (3 × 1) 벡터라는 점입니다. 행렬의 곱셈이 성립되려면 곱셈의 좌측에 있는 행렬의 열의 크기와 우측에 있는 행렬의 행의 크기가 일치해야 합니다. 현재 X_train의 행렬의 크기는 (5 × 3)이며, W 벡터의 크기는 (3 × 1)이므로 두 행렬과 벡터는 행렬곱이 가능합니다. 행렬곱으로 가설을 선언하면 아래와 같습니다.

In [26]:
hypothesis = x_train.matmul(W) + b

가설을 행렬곱으로 간단히 정의하였습니다. 이는 앞서 x_train과 w의 곱셈이 이루어지는 각 항을 전부 기재하여 가설을 선언했던 것과 대비됩니다. 이 경우, 사용자가 독립 변수 x의 수를 후에 추가적으로 늘리거나 줄이더라도 위의 가설 선언 코드를 수정할 필요가 없습니다. 이제 해야할 일은 비용 함수와 옵티마이저를 정의하고, 정해진 에포크만큼 훈련을 진행하는 일입니다. 이를 반영한 전체 코드는 다음과 같습니다.

In [46]:
x_train = torch.FloatTensor([
    [73, 80, 75],
    [93, 88, 93],
    [89, 91, 90],
    [96, 98, 100],
    [73, 66, 70]
])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer= optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = x_train.matmul(W) + b
    
    # cost 계산
    cost = torch.mean((hypothesis - y_train)**2)
    
    # cost로 H(x) 개선
    optimizer.zero_grad() # gradient를 0으로 초기화
    cost.backward() # 비용 함수를 미분하여 gradient 계산
    optimizer.step() # W와 b를 업데이트
    
#     참고) 
#     스퀴즈(Squeeze) - 1인 차원을 제거한다.
#     .detach() 를 호출하여 내용물(content)은 같지만 require_grad가 다른 새로운 Tensor를 가져옵니다
#     print(hypothesis.requires_grad) #  requires_grad: True => trainable 변수. 이는 이 변수는 학습을 통해 계속 값이 변경되는 변수임을 의미합니다.
#     test = hypothesis.detach().requires_grad # => False
#     print(test) : 

    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))
    

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371010
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo